# Retail Demo Store Messaging Workshop - Amazon Pinpoint

In this workshop we will use [Amazon Pinpoint](https://aws.amazon.com/pinpoint/) to add the ability to dynamically send personalized messages to the customers of the Retail Demo Store. We'll build out the following use-cases.

- Send new users a welcome email after they sign up for a Retail Demo Store account
- When users add items to their shopping cart but do not complete an order, send an email with a coupon code encouraging them to finish their order
- Send users an email with product recommendations from the Amazon Personalize campaign we created in the Personalization workshop

Recommended Time: 1 hour

## Prerequisites

Since this module uses Amazon Personalize to generate and associate personalized product recommendations for users, it is assumed that you have either completed the [Personalization](../1-Personalization/1.1-Personalize.ipynb) workshop or those resources have been pre-provisioned in your AWS environment. If you are unsure and attending an AWS managed event such as a workshop, check with your event lead.

## Architecture

Before diving into setting up Pinpoint to send personalize messages to our users, let's review the relevant parts of the Retail Demo Store architecture and how it uses Pinpoint to integrate with the machine learning campaigns created in Personalize.

![Retail Demo Store Pinpoint Architecture](images/retaildemostore-pinpoint-architecture.png)

### AWS Amplify & Amazon Pinpoint

The Retail Demo Store's Web UI leverages [AWS Amplify](https://aws.amazon.com/amplify/) to integrate with AWS services for authentication ([Amazon Cognito](https://aws.amazon.com/cognito/)), messaging and analytics ([Amazon Pinpoint](https://aws.amazon.com/pinpoint/)), and to keep our personalization ML models up to date ([Amazon Personalize](https://aws.amazon.com/personalize/)). AWS Amplify provides libraries for JavaScript, iOS, Andriod, and React Native for building web and mobile applications. For this workshop, we'll be focusing on how user information and events from the Retail Demo Store's Web UI are sent to Pinpoint. This is depicted as **(1)** and **(2)** in the architecture above. We'll also show how the user information and events synchronized to Pinpoint are used to create and send personalized messages.

When a new user signs up for a Retail Demo Store account, views a product, adds a product to their cart, completes an order, and so on, the relevant function is called in `AnalyticsHandler.js` in the Retail Demo Store Web UI. The new user sign up event triggers a call to the `AnalyticsHandler.identify` function where user information from Cognito is used to [update an endpoint](https://docs.aws.amazon.com/pinpoint/latest/apireference/apps-application-id-endpoints.html) in Pinpoint. In Pinpoint, an endpoint represents a destination that you can send messages to, such as a mobile device, email address, or phone number.

```javascript
// Excerpt from src/web-ui/src/analytics/AnalyticsHandler.js

export const AnalyticsHandler = {
    identify(user) {
        Vue.prototype.$Amplify.Auth.currentAuthenticatedUser().then((cognitoUser) => {
            let endpoint = {
                userId: user.id,
                optOut: 'NONE',
                userAttributes: {
                    Username: [ user.username ],
                    ProfileEmail: [ user.email ],
                    FirstName: [ user.first_name ],
                    LastName: [ user.last_name ],
                    Gender: [ user.gender ],
                    Age: [ user.age.toString() ],
                    Persona: user.persona.split("_")
                }
            }

            if (user.addresses && user.addresses.length > 0) {
                let address = user.addresses[0]
                endpoint.location = {
                    City: address.city,
                    Country: address.country,
                    PostalCode: address.zipcode,
                    Region: address.state
                }
            }

            if (cognitoUser.attributes.email) {
                endpoint.address = cognitoUser.attributes.email
                endpoint.channelType = 'EMAIL'
                Amplify.Analytics.updateEndpoint(endpoint)
            }
        })
    }
}
```

Once an `EMAIL` endpoint is created for our user, we can update attributes on that endpoint based on actions the user takes in the web UI. For example, when the user adds an item to their shopping cart, we'll set the attribute `HasShoppingCart` to `true` to indicate that this endpoint has an active shopping cart. We can also set metrics such as the number of items in the endpoint's cart. As we'll see later, we can use these attributes when building Campaigns in Pinpoint to target endpoints based on their activity in the application.

```javascript
// Excerpt from src/web-ui/src/analytics/AnalyticsHandler.js
productAddedToCart(userId, cart, product, quantity, experimentCorrelationId) {
    Amplify.Analytics.updateEndpoint({
        attributes: {
            HasShoppingCart: ['true']
        },
        metrics: {
            ItemsInCart: cart.items.length
        }
    })
}
```

When the user completes an order, we send revenue tracking events to Pinpoint, as shown below, and also update endpoint attributes and metrics. We'll see how these events, attributes, and metrics be can used later in this workshop.

```javascript
// Excerpt from src/web-ui/src/analytics/AnalyticsHandler.js
orderCompleted(user, cart, order) {
    // ...
    for (var itemIdx in order.items) {
        let orderItem = order.items[itemIdx]

        Amplify.Analytics.record({
                name: '_monetization.purchase', 
                attributes: { 
                userId: user ? user.id : null,
                cartId: cart.id,
                orderId: order.id.toString(),
                _currency: 'USD',
                _product_id: orderItem.product_id
            },
            metrics: {
                _quantity: orderItem.quantity,
                _item_price: +orderItem.price.toFixed(2)
            }
        })
    }

    Amplify.Analytics.updateEndpoint({
        attributes: {
            HasShoppingCart: ['false'],
            HasCompletedOrder: ['true']
        },
        metrics: {
            ItemsInCart: 0
        }
    })
}
```

### Integrating Amazon Pinpoint & Amazon Personalize - Pinpoint Recommenders

When building a Campaign in Amazon Pinpoint, you can associate the Pinpoint Campaign with a machine learning model, or recommender, that will be used to retrieve item recommendations for each endpoint eligible for the campaign. A recommender is linked to an Amazon Personalize Campaign. As you may recall from the [Personalization workshop](../1-Personalization/1.1-Personalize.ipynb), a Personalize Campaign only returns a list of item IDs (which represent product IDs for Retail Demo Store products). In order to turn the list of item IDs into more useful information for building a personalized email, Pinpoint supports the option to associate an AWS Lambda function to a recommender. This function is called passing information about the endpoint and the item IDs from Personalize and the function can return metadata about each item ID. Then in your Pinpoint message template you can reference the item metadata to incorporate it into your messages.

The Retail Demo Store architecture already has a Lambda function deployed to use for our Pinpoint recommender. This function calls to Retail Demo Store's Product microservice to retrieve useful information for each product (name, description, price, image URL, product URL, and so on). We will create a Pinpoint recommender in this workshop to tie it all together. This is depicted as **(3)**, **(4)**, and **(5)** in the architecture above.

## Setup

Before we can make API calls to setup Pinpoint from this notebook, we need to install and import the necessary dependencies.

### Import Dependencies

Next, let's import the dependencies we'll need for this notebook.

In [ ]:
# Import Dependencies
import boto3
import time
import json
import requests

from botocore.exceptions import ClientError

# Setup Clients
personalize = boto3.client('personalize')
ssm = boto3.client('ssm')
pinpoint = boto3.client('pinpoint')
lambda_client = boto3.client('lambda')
iam = boto3.client('iam')

# Service discovery will allow us to dynamically discover Retail Demo Store resources
servicediscovery = boto3.client('servicediscovery')

### Determine Pinpoint Application/Project

When the Retail Demo Store resources were deployed by the CloudFormation templates, a Pinpoint Application (aka Project) was automatically created with the name "retaildemostore". In order for us to interact with the application via API calls in this notebook, we need to determine the application ID.

Let's lookup our Pinpoint application using the Pinpoint API.

In [ ]:
pinpoint_app_name = 'retaildemostore'
pinpoint_app_id = None

get_apps_response = pinpoint.get_apps()
if get_apps_response['ApplicationsResponse'].get('Item'):
    for app in get_apps_response['ApplicationsResponse']['Item']:
        if app['Name'] == pinpoint_app_name:
            pinpoint_app_id = app['Id']
            break

assert pinpoint_app_id is not None, 'Retail Demo Store Pinpoint project/application does not exist'

print('Pinpoint Application ID: ' + pinpoint_app_id)

### Get Personalize Campaign ARN

Before we can create a recommender in Pinpoint, we need the Amazon Personalize Campaign ARN for the product recommendation campaign. Let's look it up in the SSM parameter store where it was set by the Personalize workshop.

In [ ]:
response = ssm.get_parameter(Name='retaildemostore-product-recommendation-campaign-arn')
personalize_campaign_arn = response['Parameter']['Value']

assert personalize_campaign_arn != 'NONE', 'Personalize Campaign ARN not initialized - run Personalization workshop'

print('Personalize Campaign ARN: ' + personalize_campaign_arn)

### Get Recommendation Customizer Lambda ARN

We also need the ARN for our Lambda function that will return product metadata for the item IDs. This function has already been deployed for you. Let's lookup our function by its name.

In [ ]:
response = lambda_client.get_function(FunctionName = 'RetailDemoStorePinpointRecommender')
lambda_function_arn = response['Configuration']['FunctionArn']
print('Recommendation customizer Lambda ARN: ' + lambda_function_arn)

### Get IAM Role for Pinpoint to access Personalize

In order for Pinpoint to access our Personalize campaign to get recommendations, we need to provide it with an IAM Role. The Retail Demo Store deployment has already created a role with the necessary policies. Let's look it up by it's role name.

In [ ]:
response = iam.get_role(RoleName = 'RetailDemoStorePinpointPersonalizeRole')
pinpoint_personalize_role_arn = response['Role']['Arn']
print('Pinpoint IAM role for Personalize: ' + pinpoint_personalize_role_arn)

## Create Pinpoint Recommender Configuration

With our environment setup and configuration info loaded, we can now create a recommender in Amazon Pinpoint. 

> We're using the Pinpoint API to create the Recommender Configuration in this workshop. You can also  create a recommeder in the AWS Console for Pinpoint under the "Machine learning models" section.

A few things to note in the recommender configuration below. 

- In the `Attributes` section, we're creating user-friendly names for the product information fields returned by our Lambda function. These names will be used in the Pinpoint console UI when designing message templates and can make it easier for template designers to select fields.
- We're using `PINPOINT_USER_ID` for the `RecommendationProviderIdType` since the endpoint's `UserId` is where we set the ID for the user in the Retail Demo Store. Since this ID is what we use to represent each user when training the recommendation models in Personalize, we need Pinpoint to use this ID as well when retrieving recommendations.
- We're limiting the number of recommendations per message to 4.

In [ ]:
response = pinpoint.create_recommender_configuration(
    CreateRecommenderConfiguration={
        'Attributes': {
            'Recommendations.Name': 'Product Name',
            'Recommendations.URL': 'Product Detail URL',
            'Recommendations.Category': 'Product Category',
            'Recommendations.Description': 'Product Description',
            'Recommendations.Price': 'Product Price',
            'Recommendations.ImageURL': 'Product Image URL'
        },
        'Description': 'Retail Demo Store Personalize recommender for Pinpoint',
        'Name': 'retaildemostore-recommender',
        'RecommendationProviderIdType': 'PINPOINT_USER_ID',
        'RecommendationProviderRoleArn': pinpoint_personalize_role_arn,
        'RecommendationProviderUri': personalize_campaign_arn,
        'RecommendationTransformerUri': lambda_function_arn,
        'RecommendationsPerMessage': 4
    }
)

recommender_id = response['RecommenderConfigurationResponse']['Id']
print('Pinpoint recommender configuration ID: ' + recommender_id)

### Verify Machine Learning Model / Recommender

If you open a web browser window/tab and browse to the Pinpoint service in the AWS console for the AWS account we're working with, you should see the ML Model / Recommender that we just created in Pinpoint.

![Pinpoint ML Model / Recommender](images/pinpoint-ml-model.png)

## Create Personalized Email Templates

With Amazon Pinpoint we can create email templates that can be used to send to groups of our users based on criteria. We'll start by creating email templates for the following use-case then step through how we target and send emails to the right users at the appropriate time.

- Welcome Email - sent to users shortly after creating a Retail Demo Store account
- Abandoned Cart Email - sent to users who leave items in their cart without completing an order
- Personalized Recommendations Email - includes recommendations from the recommeder we just created

### Load Welcome Email Templates

The first email template will be a welcome email template that is sent to new users of the Retail Demo Store after they create an account. Our templates will support both HTML and plain text formats. We'll load both formats and create the template. You can find all templates used in this workshop in the `pinpoint-templates` directory where this notebook is located. They can also be found in the Retail Demo Store source code repository.

First, let's load and display the HTML version of our welcome template.

In [ ]:
with open('pinpoint-templates/welcome-email-template.html', 'r') as html_file:
    html_template = html_file.read()
    
print('HTML Template:')
print(html_template)

Notice how we're using the mustache template tagging syntax, `{{User.UserAttributes.FirstName}}`, to display the user's first name. This will provide a nice touch of personalization to our welcome email.

Next we'll load and display the text version of our welcome email.

In [ ]:
with open('pinpoint-templates/welcome-email-template.txt', 'r') as text_file:
    text_template = text_file.read()
    
print('Text Template:')
print(text_template)

### Create Welcome Email Pinpoint Template

Now let's take our HTML and text email template source and create a template in Amazon Pinpoint. We'll use a default substitution of "there" for the user's first name attribute if it is not set for some reason. This will result in the email greeting being "Hi there,..." rather than "Hi ,..." if we don't have a value for first name.

In [ ]:
response = pinpoint.create_email_template(
    EmailTemplateRequest={
        'Subject': 'Welcome to the Retail Demo Store',
        'TemplateDescription': 'Welcome email sent to new customers',
        'HtmlPart': html_template,
        'TextPart': text_template,
        'DefaultSubstitutions': json.dumps({
            'User.UserAttributes.FirstName': 'there'
        })
    },
    TemplateName='RetailDemoStore-Welcome'
)

welcome_template_arn = response['CreateTemplateMessageBody']['Arn']
print('Welcome email template ARN: ' + welcome_template_arn)

### Load Abandoned Cart Email Templates

Next we'll create an email template that includes messaging and a coupon code for users who add items to their cart but fail to complete an order.

In [ ]:
with open('pinpoint-templates/abandoned-cart-email-template.html', 'r') as html_file:
    html_template = html_file.read()
    
print('HTML Template:')
print(html_template)

Next load the text version of our template.

In [ ]:
with open('pinpoint-templates/abandoned-cart-email-template.txt', 'r') as text_file:
    text_template = text_file.read()
    
print('Text Template:')
print(text_template)

### Create Abandoned Cart Email Template

Now we can create an email template in Pinpoint for our abandoned cart use-case.

In [ ]:
response = pinpoint.create_email_template(
    EmailTemplateRequest={
        'Subject': 'Retail Demo Store - Motivation to Complete Your Order',
        'TemplateDescription': 'Abandoned cart email template',
        'HtmlPart': html_template,
        'TextPart': text_template,
        'DefaultSubstitutions': json.dumps({
            'User.UserAttributes.FirstName': 'there'
        })   
    },
    TemplateName='RetailDemoStore-AbandonedCart'
)

abandoned_cart_template_arn = response['CreateTemplateMessageBody']['Arn']
print('Abandoned cart email template ARN: ' + abandoned_cart_template_arn)

### Load Recommendations Email Templates

Next we'll create an email template that includes recommendations from the Amazon Personalize product recommendation campaign that we created in the [Personalization workshop](../1-Personalization/1.1-Personalize.ipynb). If you haven't completed the personalization workshop, please do so now and come back to this workshop when complete.

As with the welcome email template, let's load the HTML and text formats for our template.

In [ ]:
with open('pinpoint-templates/recommendations-email-template.html', 'r') as html_file:
    html_template = html_file.read()
    
print('HTML Template:')
print(html_template)

Notice the use of several new mustache template tags in this template. For example, `{{Recommendations.Name.[0]}}` resolves to the product name of the first product recommended by Personalize. The product name came from our Lambda function which was called by Pinpoint after it called `get_recommendations` on our Personalize campaign. 

Next load the text version of our template.

In [ ]:
with open('pinpoint-templates/recommendations-email-template.txt', 'r') as text_file:
    text_template = text_file.read()
    
print('Text Template:')
print(text_template)

### Create Recommendations Email Template

This time when we create the template in Pinpoint, we'll specify the `RecommenderId` for the machine learning model (Amazon Personalize) that we created earlier.

In [ ]:
response = pinpoint.create_email_template(
    EmailTemplateRequest={
        'Subject': 'Retail Demo Store - Products Just for You',
        'TemplateDescription': 'Personalized recommendations email template',
        'RecommenderId': recommender_id,
        'HtmlPart': html_template,
        'TextPart': text_template,
        'DefaultSubstitutions': json.dumps({
            'User.UserAttributes.FirstName': 'there'
        })
    },
    TemplateName='RetailDemoStore-Recommendations'
)

recommendations_template_arn = response['CreateTemplateMessageBody']['Arn']
print('Recommendation email template ARN: ' + recommendations_template_arn)

### Verify Email Templates

If you open a web browser window/tab and browse to the Pinpoint service in the AWS console for the AWS account we're working with, you should see the message templates we just created.

![Pinpoint Message Templates](images/pinpoint-msg-templates.png)

## Enable Pinpoint Email Channel

Before we can setup Segments and Campaigns to send emails, we have to enable the email channel in Pinpoint and verify sending and receiving email addresses.

> We'll be using the Pinpoint email channel in sandbox mode. This means that Pinpoint will only send emails from and to addresses that have been verified in the Pinpoint console.

In the Pinpoint console, click on "All Projects" and then the "retaildemostore" project.

![Pinpoint Projects](images/pinpoint-projects.png)

### Email Settings

From the "retaildemostore" project page, expand "Settings" in the left navigation and then click "Email". You will see that email has not yet been enabled as a channel. Click the "Edit" button to enable Pinpoint to send emails and to verify some email addresses.

![Pinpoint Email Settings](images/pinpoint-email-setup.png)

### Verify Some Email Addresses

On the "Edit email" page, check the box to enable the email channel and enter a valid email address that you have the ability to check throughout the rest of this workshop.

![Pinpoint Verify Email Addresses](images/pinpoint-email-verify.png)

### Verify Additional Email Addresses

So that we can send an email to more than one endpoint in this workshop, verify a couple more variations of your email address.

Assuming your **valid** email address is `joe@example.com`, add a few more variations using `+` notation such as...

- `joe+1@example.com`
- `joe+2@example.com`
- `joe+3@example.com`

Just enter a variation, click the "Verify email address" button, and repeat until you've added a few more. Write down or commit to memory the variations you created--we'll need them later.

By adding these variations, we're able to create separate Retail Demo Store accounts for each email address and therefore separate endpoints in Pinpoint that we can target. Note that emails sent to the these variations should still be delivered to your same inbox.

### Check Your Inbox & Click Verification Links

Pinpoint should have sent verification emails to all of the email addresses you added above. Sign in to your email client and check your inbox for the verification emails. Once you receive the emails (it can take a few minutes), click on the verification link in **each email**. If after several minutes you don't receive the verification email or you want to use a different address, repeat the verification process above.

> Your email address(es) must be verified before we can setup Campaigns in Pinpoint.

After you click the verify link in the email sent to each variation of your email address, you should see a success page like the following.

![Email Verified](images/pinpoint-ses-success.png)

## Let's Go Shopping - Create Retail Demo Store User Accounts & Pinpoint Endpoints

Next let's create a few new user accounts in the Retail Demo Store Web UI using the email address(es) that we just verified. Based on the source code snippets we saw earlier, we know that the Retail Demo Store will create endpoints in Pinpoint for new accounts.

<div class="alert alert-info">
IMPORTANT: each Retail Demo Store account must be created in an entirely separate web browser session in order for them to be created as separate endpoints in Pinpoint. Signing out and attempting to create a new account in the same browser will NOT work. The easiest way to do this successfully is to use Google Chrome and open new Incognito windows for each new account. Alternatively, you could use multiple browser types (i.e. Chrome, Firefox, Safari, IE) and/or separate devices to create accounts such as a mobile phone or tablet.
</div>

1. Open the Retail Demo Store Web UI in an new Incognito window. If you don't already have the Web UI open or need the URL, you can find it in the "Outputs" tab for the Retail Demo Store CloudFormation stack in your AWS account. Look for the "WebURL" output field, right click on the link, and select "Open Link in Incognito Window" (Chrome only).

![CloudFormation Outputs](images/retaildemostore-cfn-outputs.png)

2. Click the "Sign In" button in the top navigation (right side) and then click on the "Create account" link in the "Sign in" form.

![Create Retail Demo Store account](images/retaildemostore-create-account.png)

3. A few seconds after creating your account you should receive an SMS with a six digit confirmation code. Enter this code on the confirmation page.

![Confirm Retail Demo Store account](images/retaildemostore-confirm-account.png)

4. Once confirmed you can sign in to your account with your user name and password. At this point you should have a endpoint in Pinpoint for this user.

5. Close your Incognito window(s).

6. Open a new Incognito window and **repeat the process for SOME (but not all) of your remaining email address variations** you verified in Pinpoint above. **As a reminder, it's important that you create each Retail Demo Store account in a separate/new Incognito window, browser application, or device. Otherwise, your accounts will overwrite the same endpoint in Pinpoint.**

<div class="alert alert-info">
Be sure to hold back one or two of your verified email addresses until after we create a welcome email campaign below so the sign up events fall within the time window of the campaign.
</div>

### Shopping Behavior

With your Retail Demo Store accounts created, perform some activities with some of your accounts. 

- For one of your users add some items to the shopping cart but do not checkout to simulate an abandoned cart scenario. 
- For another user, add some items to the cart and complete an order or two so that revenue events are sent all the way through to Pinpoint. 
- Also be sure to view a few products by clicking through to the product detail view. Select products that would indicate an affinity for a product type (e.g. shoes or electronics) so you can see how product recommendations are tailored in the product recommendations email.

## Create Pinpoint Segments

With our Recommender and message templates in place and a few test users created in the Retail Demo Store, let's turn to creating Segments in Pinpoint. After our Segments are created, we'll create some Campaigns.

1. Start by browsing to the Amazon Pinpoint service page in the AWS account where the Retail Demo Store was deployed. Click on "All Projects" and you should see the "retaildemostore" project. Click on the "retaildemostore" project and then "Segments" in the left navigation. Click on the "Create a segment" button.

![Pinpoint Segments](images/pinpoint-segments.png)

2. Then click on the "Create segment" button. We will be building a dynamic segment based on the endpoints that were automatically created when we created our Retail Demo Store user accounts. We'll include all endpoints that have an email address by adding a filter by channel type with a value of `EMAIL`. Name your segment "AllEmailUsers" and scroll down and click the "Create segment" button at the bottom of the page.

![Pinpoint Create Segment](images/pinpoint-create-segment.png)

3. Create another segment that is based on the "AllEmailUsers" segment you just created but has an additional filter on the `HasShoppingCart` endpoint attribute and has a value of `true`. This represents all users that have a shopping cart and will be used for our abandoned cart campaign. If you don't see this endpoint attribute or don't see `true` as an option, switch to another browser tab/window and add items to the shopping cart for one of your test users.

![Pinpoint Carts Segment](images/pinpoint-carts-segment.png)

## Create Campaigns

With our segments created for all users and for users with shopping carts, let's create campaigns for our welcome email, product recommendations, and abandoned cart use-cases.

### Welcome Email Campaign

Let's start with with the welcome email campaign. For the "retaildemostore" project in Pinpoint, click "Campaigns" in the left navigation and then the "Create a campaign" button.

1. For Step 1, give your campaign a name such as "WelcomeEmail", select "Standard campaign" as the campaign type, and "Email" as the channel. Click "Next" to continue.

![Pinpoint Create Campaign](images/pinpoint-create-welcome-campaign-1.png)

2. For Step 2, we will be using our "AllEmailUsers" dynamic segment. Click "Next" to continue.

![Pinpoint Create Campaign](images/pinpoint-create-welcome-campaign-2.png)

3. For Step 3, choose the "RetailDemoStore-Welcome" email template, scroll to the bottom of the page, and click "Next".

![Pinpoint Create Campaign](images/pinpoint-create-welcome-campaign-3.png)

4. For Step 4, we want the campaign to be sent when the `UserSignedUp` event occurs. Set the campaign start date to be today's date so that it begins immediately and the end date to be a few days into the future. **Be sure to adjust to your current time zone.**

![Pinpoint Create Campaign](images/pinpoint-create-welcome-campaign-4.png)

5. Scroll to the bottom of the page, click "Next".

6. Click "Launch campaign" to launch your campaign.

<div class="alert alert-info">
<strong>Given that the welcome campaign is activated based on sign up events that occur between the campaign start and end times, to test this campaign you must wait until after the camapign starts and then use one of your remaining verified email addresses to create a new Retail Demo Store account.</strong>
</div>

### Abandoned Cart Campaign

To create an abandoned cart campaign, repeat the steps you followed for the Welcome campaign above but this time select the `UsersWithCarts` segment, the `RetailDemoStore-AbandonedCart` email template, and the `Session Stop` event. This will trigger the abandoned cart email to be sent when users end their session while still having a shopping cart. Launch the campaign, wait for the campaign to start, and then close out some browser sessions for user(s) with items still in their cart. This can take some trial and error and waiting given the how browsers and devices trigger end of session events.

### Recommendations Campaign

Finally, create a recommendations campaign that targets the `AllEmailUsers` segment and uses the `RetailDemoStore-Recommendations` message template. This time, however, rather than trigger the campaign based on an event, we'll send the campaign immediately. Click "Next", launch the campaign, and check the email inbox for your test accounts after a few moments.

![Pinpoint Create Campaign](images/pinpoint-create-rec-campaign-4.png)

## Bonus - Pinpoint Journeys

With Amazon Pinpoint journeys, you can create custom experiences for your customers using an easy to use, drag-and-drop interface. When you build a journey, you choose the activities that you want to add to the journey. These activities can perform a variety of different actions, like sending an email to journey participants, waiting a defined period of time, or splitting users based on a certain action, such as when they open or click a link in an email.

Using the segments and message templates you've already created, experiment with creating a journey that guides users through a messaging experience. For example, start a journey by sending all users the Recommendations message template. Then add a pause/wait step followed by a Multivariate Split that directs users down separate paths whether they've completed an order (hint: create a `OrderCompleted` segment), opened the Recommendations email, or done nothing. Perhaps users who completed an order might receive a message asking them to refer a friend to the Retail Demo Store and users who just opened the email might be sent a message with a coupon to motivate them to get shopping (you'll need to create a new message templates for these).

## Workshop Complete

Congratulations! You have completed the Retail Demo Store Pinpoint Workshop.

### Cleanup

If you launched the Retail Demo Store in your personal AWS account **AND** you're done with all workshops & your evaluation of the Retail Demo Store, you can remove all provisioned AWS resources and data by deleting the CloudFormation stack you used to deploy the Retail Demo Store. Although deleting the CloudFormation stack will delete the entire "retaildemostore" project in Pinpoint, including all endpoint data, it will not delete resources we created directly in this workshop (i.e. outside of the "retaildemostore" Pinpoint project). The following cleanup steps will remove the resources we created outside the "retaildemostore" Pinpoint project.

> If you are participating in an AWS managed event such as a workshop and using an AWS provided temporary account, you can skip the following cleanup steps unless otherwise instructed.

#### Delete Recommeder Configuration

In [ ]:
response = pinpoint.delete_recommender_configuration(RecommenderId=recommender_id)
print(json.dumps(response, indent=2))

#### Delete Email Message Templates

In [ ]:
response = pinpoint.delete_email_template(TemplateName='RetailDemoStore-Welcome')
print(json.dumps(response, indent=2))

In [ ]:
response = pinpoint.delete_email_template(TemplateName='RetailDemoStore-AbandonedCart')
print(json.dumps(response, indent=2))

In [ ]:
response = pinpoint.delete_email_template(TemplateName='RetailDemoStore-Recommendations')
print(json.dumps(response, indent=2))

Other resources allocated for the Retail Demo Store will be deleted when the CloudFormation stack is deleted.

End of workshop